# Build LLM Agent combining Reasoning and Action (ReAct) framework using LangChain - Ashish Kumar Jain

### ReAct is technique which enable LLMs to do reasoning and take task specific actions. It combines chain of thought reasoning with action planning.It enables LLMs to generate reasoning traces and task-specific actions, leveraging the synergy between them. This approach demonstrates superior performance over baselines in various tasks, overcoming issues like hallucination and error propagation.

### In the notebook we will use OPENAI ChatGPT model. We can integrate the ChatGPT model in our application using API provided by OPENAI. For accessing the API, we need to register with OPENAI and get the api key for the same. 
#### We will use Langchain framework and python code for illustration purpose. You can use this code for your applications.You can also refer Langchain site for more code references. You also need to install below required python library to run the code.

In [1]:
#pip install openai
#pip install langchain
#pip install wikipedia

#### First we need to load the key from environment variable and set into openai. You will get this key while registering with OPENAI.

In [2]:
import openai
import os
openai.api_key  = os.getenv('OPENAI_API_KEY')

#### Langchain has concept of Agent. The core idea of agent is to use a language model to choose a sequence of actions to take. In agent, a language model is used as a reasoning engine to determine which actions to take and in which order. Langchain has inbuilt agent called DocStore Explorer agent whose functionality aligns with the original ReAct research paper, for the Wikipedia use case. 

#### DocStoreExplorer agent interacts with a document storage system (like Wikipedia), using two specific tools , a Search tool and a Lookup tool. The Search tool is responsible for locating a document, whereas the Lookup tool retrieves a term from the most recently discovered document. We can initialize this doc store with Wikipedia as a document store.

In [3]:
from langchain.agents.react.base import DocstoreExplorer
from langchain.docstore import Wikipedia
docstore = DocstoreExplorer(Wikipedia())

#### Another concept which Langchain provides is called tools. Tools are interfaces that an agent can use to interact with the outer world. We can create our own tool for Wikipedia API. In the tool specification, we can specify which function agent need to call for interaction with outer world.

In [4]:
from langchain.agents import Tool 
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="useful for when you need to ask with search",
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="useful for when you need to ask with lookup",
    ),
]

#### Lets have GPT-4 model as a LLM for our application.

In [5]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4")

#### Lets initialize agent with tools, llm and agent with all the information.

In [6]:
from langchain.agents import AgentType, initialize_agent
react = initialize_agent(tools, llm, agent=AgentType.REACT_DOCSTORE, verbose=True)

#### We can pass our question to our ReAct agent. Behind the scene, Agent will do interaction with OPENAI model using API. It will also pass tools information (description of functions ) to the model as an additional arguments (in case of OPENAI its called OPENAI functions) with prompt. If model want to take search or lookup actions then it returns that function description with function arguments (if any) to the agent. Then agent call that function using the tool and call Model again with that information. It will do this in iterative manner as explained above. All this magic is done by agent for us.

In [7]:
question = "What profession does Amitabh Bachchan and Shahrukh Khan have in common?"
result = react.run(question)



> Entering new AgentExecutor chain...
Thought: I need to search Amitabh Bachchan and Shahrukh Khan, find their professions, then find the profession they have in common.
Action: Search[Amitabh Bachchan]
Observation: Amitabh Bachchan (pronounced [əmɪˈt̪ɑːbʱ ˈbətːʃən]; born as Amitabh Srivastava; 11 October 1942) is an Indian actor, film producer, television host, occasional playback singer, and former politician actively engaged in Hindi cinema. With a cinematic journey spanning over five decades, he has played pivotal roles in over 200 films. Widely recognised as one of the most accomplished and influential actors in the history of Indian cinema. Bachchan is often hailed as the Shahenshah of Bollywood, Sadi Ke Mahanayak (translated as "Greatest actor of the century" in Hindi), Star of the Millennium, or simply Big B. His dominance in the Indian film industry during the 1970s–80s led the French director François Truffaut to describe it as a "one-man industry".Bachchan was born in 1942

#### We can finally get the answer

In [8]:
result

'actor and film producer'